In [2]:
# Check python version and report it
import sys
sys.version

In [ ]:
# General imports
import os

Plot to external windows from the notebook by using Qt as the backend:
    '%pylab qt'
or make inline plots in the notebook
    '%pylab inline'

In [4]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


#### Setup the python path is necessary

This is the location of your pyrpl repo. It is only necessary if you are doing development and did not install pyrpl. 

For reference, you can use 
    'pip install --editable .' 
in your git repo to add it to your python path automatically.

In [ ]:
if 0:   # change if necessary
    pyrpl_home = os.path.join('G:/', 'Workshop', 'projects', 'pyrpl')  # path to git repo ('pyrpl_home/pyrpl' is the package)
    
    # Add pyrpl to your local python path
    sys.path.append(pyrpl_home)
else:
    pyrpl_home = os.path.abspath(os.path.join(os.path.dirname(pyrpl.__file__), '..'))

Test the import of pyrpl

In [ ]:
from pyrpl import Pyrpl

##### Test importing asynchronous methods. They are extremely dependent on python version due to fundamental changes in how asyncio is done in python 3.6+.

In [4]:
from pyrpl.async_utils import sleep, wait, ensure_future, sleep_async
from asyncio import iscoroutine, Future

#### Quickly test the asyncio features and functions 

In [33]:
sleep_async(1)

<coroutine object sleep_async at 0x7fd95600d710>

In [17]:
isinstance(ensure_future(sleep_async(1)), Future)

True

In [2]:
t = ensure_future(sleep_async(1))

In [5]:
isinstance(t, Future)

True

In [8]:
t

<Task finished coro=<sleep_async() done, defined at /Users/samuel/Documents/GitHub/pyrpl/pyrpl/async_utils.py:66> result=None>

In [9]:
ensure_future(sleep_async(1))

<Task pending coro=<sleep_async() running at /Users/samuel/Documents/GitHub/pyrpl/pyrpl/async_utils.py:66>>

In [29]:
Future

_asyncio.Future

In [32]:
t.__class__

_asyncio.Task

In [31]:
ensure_future.__doc__

'\n    Schedules the task described by the coroutine. Deals properly with\n    IPython kernel integration.\n    '

In [10]:
from asyncio.tasks import Task

In [13]:
isinstance(Task(sleep_async(1)), Future)

True

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<sleep_async() done, defined at /Users/samuel/Documents/GitHub/pyrpl/pyrpl/async_utils.py:66> exception=RuntimeError('Task <Task pending coro=<sleep_async() running at /Users/samuel/Documents/GitHub/pyrpl/pyrpl/async_utils.py:71>> got Future <Future pending> attached to a different loop')>
Traceback (most recent call last):
  File "/Users/samuel/Documents/GitHub/pyrpl/pyrpl/async_utils.py", line 71, in sleep_async
    await asyncio.sleep(time_s, loop=LOOP)
  File "/Users/samuel/opt/anaconda3/envs/py37/lib/python3.7/asyncio/tasks.py", line 595, in sleep
    return await future
RuntimeError: Task <Task pending coro=<sleep_async() running at /Users/samuel/Documents/GitHub/pyrpl/pyrpl/async_utils.py:71>> got Future <Future pending> attached to a different loop


##### If pyrpl successfully imports, then make a connection to a red-pitaya to test it

In [ ]:
# This is your red pitaya IP address or host-name.
if 1:
    # Code testing no active connection desired
    rphost = "redpitaya"
else:
    # Testing with a real device
    rphost = '192.168.44.111'

if 0:  # If you like you can add your red-pitaya host as an environment variable
    os.environ["REDPITAYA_HOSTNAME"] = rphost

Instantiate a Pyrpl object

Note that passing an empty config string is identical with passing nothing as it is replaced with the default, config = None

In [5]:
p = Pyrpl(config="")

INFO:pyrpl.redpitaya:Successfully connected to Redpitaya with hostname redpitaya.


These classes / functions allow running jupyter notebooks remotely

#### Run the test notebook externally

In [ ]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from nbconvert.preprocessors.execute import CellExecutionError

class MyExecutePreprocessor(ExecutePreprocessor):
    def preprocess_cell(self, cell, resources, cell_index):
        if cell.source.startswith("#no-test"):
            return cell, resources
        if cell.source.startswith("#define-hostname"):
            # replace hostname by unittest hostname
            if defaultparameters["hostname"] is not None:
                cell.source = 'HOSTNAME = ' + defaultparameters["hostname"]
            if 'REDPITAYA_HOSTNAME' in os.environ:
                cell.source = 'HOSTNAME = "' + os.environ[
                    "REDPITAYA_HOSTNAME"] + '"'
        return super(MyExecutePreprocessor, self).preprocess_cell(cell,\
                resources, cell_index)


def _notebook_run(path):
    """
    Execute a notebook via nbconvert and collect output.
    :returns (parsed nb object, execution errors)
    """
    kernel_name = 'python%d' % sys.version_info[0]
    errors = []


    with open(path) as f:
        nb = nbformat.read(f, as_version=4)
        nb.metadata.get('kernelspec', {})['name'] = kernel_name
        ep = MyExecutePreprocessor(kernel_name=kernel_name, timeout=65) #,
        # allow_errors=True
        #ep.start_new_kernel()
        try:
            ep.preprocess(nb, resources={'metadata': {'path': ""}})
        except (CellExecutionError, TimeoutError) as e:
          if hasattr(e, 'traceback') and "SKIP" in e.traceback:
                print(str(e.traceback).split("\n")[-2])
          else:
            raise e
    return nb, errors

In [ ]:
# filename = r"/Users/samuel/Documents/GitHub/pyrpl/pyrpl/test/test_ipython_notebook/test_notebook.ipynb"
filename = r"" + os.path.join(pyrpl_home, 'pyrpl', 'test', 'test_ipython_notebook', 'test_notebook.ipynb')

In [ ]:
_notebook_run(filename)